# Demo Minggu 4: Filter Digital (*Digital Filters*)
## Pengolahan Sinyal Medis -- Universitas Indonesia

---

### Tujuan Pembelajaran

Setelah menyelesaikan demo ini, mahasiswa diharapkan mampu:

1. Menjelaskan mengapa filter digital diperlukan dalam pengolahan sinyal biomedis.
2. Memahami konsep **respons frekuensi** (*frequency response*) dan cara memvisualisasikannya.
3. Merancang dan menerapkan filter **IIR Butterworth** (lowpass, highpass, bandpass, notch).
4. Merancang dan menerapkan filter **FIR** (*Finite Impulse Response*) menggunakan windowing.
5. Membandingkan karakteristik IIR vs FIR untuk aplikasi biomedis.
6. Membangun **pipeline pra-pemrosesan ECG** (*ECG preprocessing pipeline*) secara lengkap.
7. Mengekstraksi **pita frekuensi EEG** (*EEG frequency bands*) menggunakan bandpass filter.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig

# Library kursus
from medsinyal.io import load_synthetic
from medsinyal.viz import plot_signal, plot_signals, plot_ecg
from medsinyal.filters import (
    bandpass_filter,
    lowpass_filter,
    highpass_filter,
    notch_filter,
    remove_baseline_wander,
    moving_average,
    design_fir_bandpass,
    apply_fir,
)

# Konfigurasi plot global
plt.rcParams['figure.figsize'] = (13, 4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.alpha'] = 0.3

print('Semua library berhasil dimuat.')

---
## 1. Pendahuluan: Mengapa Filter Digital?

Sinyal biomedis mentah (*raw signal*) hampir selalu mengandung komponen yang tidak diinginkan.
Filter digital memungkinkan kita **memilih secara selektif** komponen frekuensi mana yang
dipertahankan dan mana yang dihilangkan.

### Keterbatasan Moving Average

Pada Minggu 2 kita sudah melihat bahwa *moving average* -- meski sederhana -- tidak selektif
terhadap frekuensi. Ia menghaluskan semua komponen, termasuk fitur klinis penting seperti
kompleks QRS pada ECG.

### Jenis-Jenis Filter Digital

| Jenis Filter | *Pass Band* | *Stop Band* | Aplikasi Biomedis |
|---|---|---|---|
| **Lowpass** (*LPF*) | $0 \leq f \leq f_c$ | $f > f_c$ | Smoothing, anti-aliasing |
| **Highpass** (*HPF*) | $f \geq f_c$ | $f < f_c$ | Hapus *baseline wander* ECG (0.5 Hz) |
| **Bandpass** (*BPF*) | $f_L \leq f \leq f_H$ | $f < f_L$ dan $f > f_H$ | ECG diagnostik (0.5--40 Hz) |
| **Bandstop / Notch** (*BSF*) | $f < f_L$ dan $f > f_H$ | $f_L \leq f \leq f_H$ | Hapus interferensi 50 Hz |

### Aplikasi Klinis Filter

| Sinyal | Masalah | Solusi Filter |
|---|---|---|
| ECG | *Baseline wander* akibat pernapasan | Highpass 0.5 Hz |
| ECG | Interferensi jala listrik | Notch 50 Hz |
| ECG diagnostik | Semua noise frekuensi tinggi | Bandpass 0.5--40 Hz |
| ECG monitoring ICU | Rentang lebih lebar | Bandpass 0.5--150 Hz |
| EEG | Ekstraksi pita delta | Bandpass 0.5--4 Hz |
| EEG | Ekstraksi pita alpha | Bandpass 8--13 Hz |

In [ ]:
# Memuat data sintetis yang akan digunakan sepanjang demo
data_ecg = load_synthetic('synthetic_ecg')
data_eeg = load_synthetic('synthetic_eeg')

t_ecg      = data_ecg['t']
fs_ecg     = float(data_ecg['fs'])
ecg_clean  = data_ecg['ecg_clean']
ecg_noisy  = data_ecg['ecg_noisy']
r_peaks    = data_ecg['r_peak_indices']

t_eeg      = data_eeg['t']
fs_eeg     = float(data_eeg['fs'])
eeg_noisy  = data_eeg['eeg_noisy']
eeg_clean  = data_eeg['eeg_clean']

print(f'ECG: fs={fs_ecg} Hz, durasi={t_ecg[-1]:.1f} s, n={len(ecg_clean)} sampel')
print(f'EEG: fs={fs_eeg} Hz, durasi={t_eeg[-1]:.1f} s, n={len(eeg_clean)} sampel')

# Tampilkan sinyal ECG bersih vs noisy sebagai motivasi
fig, axes = plt.subplots(2, 1, figsize=(13, 6), sharex=True)
plot_ecg(t_ecg, ecg_clean, title='ECG Bersih (Referensi)', ax=axes[0])
plot_ecg(t_ecg, ecg_noisy,
         title='ECG dengan Noise (Baseline Wander + Interferensi 50 Hz + Random Noise)',
         ax=axes[1])
fig.tight_layout()
plt.show()

# Demonstrasi keterbatasan moving average
ecg_ma = moving_average(ecg_noisy, window_size=25)  # ~50 ms pada fs=500 Hz
fig, ax = plt.subplots(figsize=(13, 3))
ax.plot(t_ecg, ecg_clean, 'b-', linewidth=0.8, alpha=0.5, label='ECG Bersih')
ax.plot(t_ecg, ecg_ma,    'r-', linewidth=1.0, alpha=0.8, label='Moving Average (M=25)')
ax.set_title('Keterbatasan Moving Average: QRS Complex Terdistorsi')
ax.set_xlabel('Waktu (s)')
ax.set_ylabel('Amplitudo (mV)')
ax.legend()
ax.set_xlim(0, 4)
plt.tight_layout()
plt.show()
print('Perhatikan: puncak R menjadi lebih rendah dan melebar -- informasi klinis terdistorsi.')

---
## 2. Respons Frekuensi (*Frequency Response*)

Karakteristik filter digital sepenuhnya ditentukan oleh **respons frekuensinya** $H(f)$.

### Fungsi Transfer (*Transfer Function*)

Untuk filter IIR dalam domain-$z$:

$$H(z) = \frac{B(z)}{A(z)} = \frac{b_0 + b_1 z^{-1} + \cdots + b_M z^{-M}}{1 + a_1 z^{-1} + \cdots + a_N z^{-N}}$$

Respons frekuensi diperoleh dengan mengevaluasi $H(z)$ pada lingkaran satuan
($z = e^{j\omega}$):

$$H(e^{j\omega}) = |H(e^{j\omega})| \cdot e^{j\angle H(e^{j\omega})}$$

- **Respons magnitudo** $|H(f)|$: menunjukkan seberapa besar setiap komponen frekuensi
  diperkuat atau dilemahkan.
- **Respons fase** $\angle H(f)$: menunjukkan pergeseran fase yang dialami setiap komponen
  frekuensi.

### Efek Orde Filter (*Filter Order*)

Semakin tinggi orde filter, semakin **curam** transisi antara *passband* dan *stopband*
(*rolloff* lebih tajam).

In [ ]:
# Visualisasi respons frekuensi Butterworth lowpass pada berbagai orde
fs_demo  = 500.0   # Hz
fc_demo  = 40.0    # Hz -- frekuensi cutoff
orders   = [2, 4, 8]
colors   = ['tab:blue', 'tab:orange', 'tab:green']

fig, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(13, 7), sharex=True)
fig.suptitle(
    f'Respons Frekuensi Filter Butterworth Lowpass (fc = {fc_demo} Hz, fs = {fs_demo} Hz)',
    fontsize=13, fontweight='bold'
)

for order, color in zip(orders, colors):
    # Rancang filter Butterworth menggunakan scipy secara langsung
    b, a = sig.butter(order, fc_demo / (fs_demo / 2), btype='low')
    # freqz menghitung respons frekuensi digital
    w, h = sig.freqz(b, a, worN=4096, fs=fs_demo)

    mag_db    = 20 * np.log10(np.abs(h) + 1e-12)
    phase_deg = np.unwrap(np.angle(h)) * 180 / np.pi

    ax_mag.plot(w, mag_db, color=color, linewidth=1.5, label=f'Orde {order}')
    ax_phase.plot(w, phase_deg, color=color, linewidth=1.5, label=f'Orde {order}')

# Garis referensi
ax_mag.axvline(fc_demo, color='red', linestyle='--', linewidth=1, label=f'fc = {fc_demo} Hz')
ax_mag.axhline(-3, color='gray', linestyle=':', linewidth=1, label='-3 dB')
ax_mag.set_ylabel('Magnitudo (dB)')
ax_mag.set_ylim(-80, 5)
ax_mag.legend(loc='lower left')
ax_mag.set_title('Respons Magnitudo |H(f)|')

ax_phase.axvline(fc_demo, color='red', linestyle='--', linewidth=1)
ax_phase.set_ylabel('Fase (derajat)')
ax_phase.set_xlabel('Frekuensi (Hz)')
ax_phase.set_title('Respons Fase -- IIR memiliki fase non-linear')
ax_phase.legend(loc='lower left')
ax_phase.set_xlim(0, fs_demo / 2)

plt.tight_layout()
plt.show()

print('Pengamatan:')
print('  - Orde lebih tinggi -> transisi passband-stopband lebih curam (rolloff lebih tajam).')
print('  - Orde lebih tinggi -> distorsi fase di sekitar cutoff lebih besar.')
print('  - Di -3 dB, semua orde bertemu tepat di fc (sifat khas Butterworth).')

---
## 3. Filter IIR: Butterworth

**IIR** (*Infinite Impulse Response*) adalah filter rekursif yang menggunakan
umpan balik (*feedback*). Respons impulsnya secara teori berlangsung tak terhingga.

### Persamaan Beda (*Difference Equation*)

$$y[n] = \sum_{k=0}^{M} b_k \, x[n-k] - \sum_{k=1}^{N} a_k \, y[n-k]$$

Suku $-\sum a_k y[n-k]$ adalah bagian **rekursif** (umpan balik dari output sebelumnya).

### Filter Butterworth

Butterworth dirancang agar respons magnitude **maksimum datar** (*maximally flat*) di passband:

$$|H(f)|^2 = \frac{1}{1 + \left(\dfrac{f}{f_c}\right)^{2N}}$$

Sifat utama:
- Tidak ada *ripple* di passband maupun stopband.
- Rolloff: $-20N$ dB/dekade ($N$ = orde filter).
- Digunakan luas di peralatan ECG dan EEG klinis.

> **Catatan implementasi**: `scipy.signal.filtfilt` menerapkan filter dua kali (maju-mundur)
> sehingga menghasilkan **fase nol** (*zero-phase*) -- tidak ada distorsi fase pada sinyal
> output. Ini sangat penting untuk diagnosis klinis.

In [ ]:
# Rancang bandpass filter Butterworth secara langsung dengan scipy
# kemudian visualisasikan respons frekuensinya
lowcut_ecg  = 0.5   # Hz
highcut_ecg = 40.0  # Hz
order_iir   = 4

b_iir, a_iir = sig.butter(
    order_iir,
    [lowcut_ecg / (fs_ecg / 2), highcut_ecg / (fs_ecg / 2)],
    btype='band'
)
w_iir, h_iir = sig.freqz(b_iir, a_iir, worN=4096, fs=fs_ecg)

fig, ax = plt.subplots(figsize=(13, 4))
ax.plot(w_iir, 20 * np.log10(np.abs(h_iir) + 1e-12), 'b-', linewidth=1.5,
        label=f'Butterworth Bandpass Orde {order_iir}')
ax.axvline(lowcut_ecg,  color='red',   linestyle='--', linewidth=1,
           label=f'fL = {lowcut_ecg} Hz')
ax.axvline(highcut_ecg, color='green', linestyle='--', linewidth=1,
           label=f'fH = {highcut_ecg} Hz')
ax.axhline(-3, color='gray', linestyle=':', linewidth=1, label='-3 dB')
ax.set_xlim(0, 120)
ax.set_ylim(-60, 5)
ax.set_xlabel('Frekuensi (Hz)')
ax.set_ylabel('Magnitudo (dB)')
ax.set_title(f'Respons Frekuensi IIR Butterworth Bandpass ({lowcut_ecg}--{highcut_ecg} Hz)')
ax.legend()
plt.tight_layout()
plt.show()

# Terapkan IIR bandpass pada ECG noisy menggunakan medsinyal
ecg_iir   = bandpass_filter(ecg_noisy, lowcut_ecg, highcut_ecg, fs_ecg, order=order_iir)

rmse_noisy = np.sqrt(np.mean((ecg_noisy - ecg_clean) ** 2))
rmse_iir   = np.sqrt(np.mean((ecg_iir   - ecg_clean) ** 2))

fig, axes = plt.subplots(3, 1, figsize=(13, 9), sharex=True)
plot_ecg(t_ecg, ecg_clean, title='ECG Bersih (Referensi)', ax=axes[0])
plot_ecg(t_ecg, ecg_noisy,
         title=f'ECG Noisy  |  RMSE = {rmse_noisy:.4f} mV', ax=axes[1])
plot_ecg(t_ecg, ecg_iir,
         title=(
             f'ECG setelah IIR Butterworth Bandpass '
             f'({lowcut_ecg}--{highcut_ecg} Hz)  |  RMSE = {rmse_iir:.4f} mV'
         ),
         ax=axes[2])
for ax in axes:
    ax.set_xlim(0, 5)
fig.tight_layout()
plt.show()

print(f'RMSE sebelum filter : {rmse_noisy:.4f} mV')
print(f'RMSE setelah IIR    : {rmse_iir:.4f} mV')
print(f'Perbaikan           : {(1 - rmse_iir/rmse_noisy)*100:.1f}%')

---
## 4. Notch Filter -- Menghilangkan Interferensi 50 Hz

**Notch filter** (atau *band-stop filter* sempit) dirancang untuk menghilangkan satu
frekuensi spesifik sambil membiarkan semua frekuensi lain lewat.

Interferensi jala listrik (*power line interference*) pada **50 Hz** (di Indonesia dan Eropa)
atau **60 Hz** (Amerika Utara) adalah salah satu sumber noise paling umum dalam rekaman
ECG dan EEG.

### Parameter Kunci: *Quality Factor* $Q$

$$Q = \frac{f_0}{\Delta f}$$

di mana $f_0$ adalah frekuensi notch dan $\Delta f$ adalah lebar pita attenuasi:
- $Q$ besar (misalnya 30) $\to$ notch sangat sempit $\to$ hanya menghilangkan
  frekuensi tepat di $f_0$.
- $Q$ kecil $\to$ notch lebih lebar $\to$ lebih banyak frekuensi sekitar $f_0$
  ikut terhapus.

In [ ]:
# Terapkan notch filter 50 Hz menggunakan medsinyal
ecg_notched = notch_filter(ecg_noisy, freq=50.0, fs=fs_ecg, quality_factor=30.0)

# Hitung FFT sebelum dan sesudah notch untuk konfirmasi
N_fft = len(ecg_noisy)
freqs        = np.fft.rfftfreq(N_fft, d=1.0 / fs_ecg)
fft_noisy   = np.abs(np.fft.rfft(ecg_noisy))   / N_fft * 2
fft_notched = np.abs(np.fft.rfft(ecg_notched)) / N_fft * 2

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Notch Filter 50 Hz pada ECG', fontsize=13, fontweight='bold')

# Domain waktu: sebelum
axes[0, 0].plot(t_ecg, ecg_noisy, 'r-', linewidth=0.8)
axes[0, 0].set_title('ECG Noisy -- Domain Waktu')
axes[0, 0].set_xlim(0, 3)
axes[0, 0].set_ylabel('Amplitudo (mV)')
axes[0, 0].set_xlabel('Waktu (s)')

# Domain waktu: sesudah
axes[0, 1].plot(t_ecg, ecg_notched, 'g-', linewidth=0.8)
axes[0, 1].set_title('ECG setelah Notch 50 Hz -- Domain Waktu')
axes[0, 1].set_xlim(0, 3)
axes[0, 1].set_ylabel('Amplitudo (mV)')
axes[0, 1].set_xlabel('Waktu (s)')

# FFT: sebelum
axes[1, 0].plot(freqs, fft_noisy, 'r-', linewidth=0.8)
axes[1, 0].axvline(50, color='orange', linestyle='--', linewidth=1.5, label='50 Hz')
axes[1, 0].set_title('Spektrum FFT -- Sebelum Notch')
axes[1, 0].set_xlim(0, 120)
axes[1, 0].set_xlabel('Frekuensi (Hz)')
axes[1, 0].set_ylabel('Magnitudo')
axes[1, 0].legend()

# FFT: sesudah
axes[1, 1].plot(freqs, fft_notched, 'g-', linewidth=0.8)
axes[1, 1].axvline(50, color='orange', linestyle='--', linewidth=1.5,
                   label='50 Hz (dihilangkan)')
axes[1, 1].set_title('Spektrum FFT -- Setelah Notch')
axes[1, 1].set_xlim(0, 120)
axes[1, 1].set_xlabel('Frekuensi (Hz)')
axes[1, 1].set_ylabel('Magnitudo')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Konfirmasi numerik: amplitudo di 50 Hz sebelum dan sesudah
idx_50 = np.argmin(np.abs(freqs - 50.0))
print(f'Amplitudo pada 50 Hz sebelum notch : {fft_noisy[idx_50]:.5f}')
print(f'Amplitudo pada 50 Hz setelah notch : {fft_notched[idx_50]:.5f}')
print(f'Reduksi amplitudo 50 Hz            : {(1 - fft_notched[idx_50]/fft_noisy[idx_50])*100:.1f}%')

---
## 5. Filter FIR (*Finite Impulse Response*)

**FIR** (*Finite Impulse Response*) adalah filter non-rekursif -- outputnya hanya bergantung
pada input saat ini dan input sebelumnya, tanpa umpan balik:

$$y[n] = \sum_{k=0}^{M-1} h[k] \cdot x[n-k]$$

di mana $h[k]$ adalah **koefisien filter** (juga disebut *tap*).

### Keunggulan Filter FIR

| Properti | FIR | IIR |
|---|---|---|
| **Stabilitas** | Selalu stabil | Bisa tidak stabil |
| **Respons fase** | Linear (tidak ada distorsi fase) | Non-linear |
| **Jumlah koefisien** | Banyak (puluhan hingga ratusan) | Sedikit (orde rendah) |
| **Biaya komputasi** | Lebih tinggi | Lebih rendah |
| **Desain** | Lebih fleksibel | Berdasarkan prototipe analog |

### Metode Desain: *Windowing*

1. Tentukan respons impuls ideal $h_d[n]$ (dalam domain frekuensi = kotak sempurna).
2. Kalikan dengan fungsi jendela $w[n]$ (misalnya Hamming) untuk membatasi panjang filter:

$$h[n] = h_d[n] \cdot w[n], \quad n = 0, 1, \ldots, M-1$$

Fungsi `scipy.signal.firwin` mengimplementasikan pendekatan ini secara otomatis.

In [ ]:
# Rancang FIR bandpass (0.5--40 Hz) menggunakan scipy.signal.firwin
numtaps_fir = 101   # jumlah tap -- harus ganjil untuk simetri
fir_coeffs  = sig.firwin(
    numtaps_fir,
    [lowcut_ecg, highcut_ecg],
    pass_zero=False,
    fs=fs_ecg,
    window='hamming'
)

# Hitung respons frekuensi FIR
w_fir, h_fir = sig.freqz(fir_coeffs, worN=4096, fs=fs_ecg)

# Bandingkan respons magnitudo dan fase FIR vs IIR
fig, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(13, 7), sharex=True)
fig.suptitle(
    f'Perbandingan Respons Frekuensi: IIR Butterworth vs FIR Hamming\n'
    f'(Bandpass {lowcut_ecg}--{highcut_ecg} Hz)',
    fontsize=13, fontweight='bold'
)

# Respons magnitudo
ax_mag.plot(w_iir, 20 * np.log10(np.abs(h_iir) + 1e-12),
            'b-', linewidth=2, label=f'IIR Butterworth Orde {order_iir}')
ax_mag.plot(w_fir, 20 * np.log10(np.abs(h_fir) + 1e-12),
            color='orange', linewidth=1.5, linestyle='--',
            label=f'FIR Hamming ({numtaps_fir} tap)')
ax_mag.axvline(lowcut_ecg,  color='red',   linestyle=':', linewidth=1,
               label=f'fL={lowcut_ecg} Hz')
ax_mag.axvline(highcut_ecg, color='green', linestyle=':', linewidth=1,
               label=f'fH={highcut_ecg} Hz')
ax_mag.axhline(-3, color='gray', linestyle=':', linewidth=1, label='-3 dB')
ax_mag.set_xlim(0, 120)
ax_mag.set_ylim(-80, 5)
ax_mag.set_ylabel('Magnitudo (dB)')
ax_mag.legend(loc='lower left')
ax_mag.set_title('Respons Magnitudo')

# Respons fase
phase_iir = np.unwrap(np.angle(h_iir)) * 180 / np.pi
phase_fir = np.unwrap(np.angle(h_fir)) * 180 / np.pi
ax_phase.plot(w_iir, phase_iir, 'b-', linewidth=2, label='IIR (non-linear)')
ax_phase.plot(w_fir, phase_fir, color='orange', linewidth=1.5, linestyle='--',
              label='FIR (linear -- tidak ada distorsi fase)')
ax_phase.set_xlim(0, 120)
ax_phase.set_ylabel('Fase (derajat)')
ax_phase.set_xlabel('Frekuensi (Hz)')
ax_phase.set_title('Respons Fase')
ax_phase.legend()

plt.tight_layout()
plt.show()

# Terapkan FIR pada ECG noisy menggunakan medsinyal.apply_fir
# design_fir_bandpass dari medsinyal setara dengan firwin di atas
fir_coeffs_ms = design_fir_bandpass(lowcut_ecg, highcut_ecg, fs_ecg, numtaps=numtaps_fir)
ecg_fir       = apply_fir(ecg_noisy, fir_coeffs_ms)

rmse_fir = np.sqrt(np.mean((ecg_fir - ecg_clean) ** 2))
print(f'FIR -- jumlah koefisien: {len(fir_coeffs_ms)}')
print(f'FIR -- RMSE vs clean   : {rmse_fir:.4f} mV')
print(f'IIR -- RMSE vs clean   : {rmse_iir:.4f} mV')

---
## 6. Perbandingan IIR vs FIR

Memilih antara IIR dan FIR bergantung pada kebutuhan aplikasi klinis.

| Kriteria | IIR (Butterworth) | FIR (Hamming) |
|---|---|---|
| **Stabilitas** | Perlu diverifikasi | Selalu stabil |
| **Fase** | Non-linear (*phase distortion*) | Linear (tidak ada distorsi) |
| **Jumlah koefisien** | Rendah (orde 4--8) | Tinggi (50--500 tap) |
| **Biaya komputasi** | Rendah | Lebih tinggi |
| **Ripple passband** | Tidak ada (Butterworth) | Ada (terkontrol) |
| **Pilihan untuk** | Real-time, embedded system | Analisis offline, *phase-critical* |

> **Catatan klinis**: Untuk analisis gelombang P dan T pada ECG, distorsi fase sangat
> kritis karena dapat menggeser waktu onset dan offset. Filter FIR (atau `filtfilt` IIR)
> lebih disarankan.

In [ ]:
# Bandingkan hasil IIR vs FIR pada ECG secara visual dan kuantitatif
fig, axes = plt.subplots(4, 1, figsize=(13, 12), sharex=True)
fig.suptitle('Perbandingan IIR vs FIR pada ECG Noisy', fontsize=13, fontweight='bold')

plot_ecg(t_ecg, ecg_clean,  title='(Referensi) ECG Bersih', ax=axes[0])
plot_ecg(t_ecg, ecg_noisy,  title='ECG Noisy (Input)', ax=axes[1])
plot_ecg(t_ecg, ecg_iir,
         title=f'IIR Butterworth Bandpass Orde {order_iir}  |  RMSE={rmse_iir:.4f} mV',
         ax=axes[2])
plot_ecg(t_ecg, ecg_fir,
         title=f'FIR Hamming ({numtaps_fir} tap)  |  RMSE={rmse_fir:.4f} mV',
         ax=axes[3])

for ax in axes:
    ax.set_xlim(0, 5)

plt.tight_layout()
plt.show()

# Ringkasan kuantitatif
print('\n--- Ringkasan Kuantitatif (RMSE vs ECG Bersih) ---')
print(f'{"Metode":<35}  {"RMSE (mV)":>10}  {"Perbaikan":>10}')
print('-' * 58)
print(f'{"ECG Noisy (tanpa filter)":<35}  {rmse_noisy:>10.4f}  {"--":>10}')
print(
    f'{f"IIR Butterworth Orde {order_iir}":<35}  '
    f'{rmse_iir:>10.4f}  {(1-rmse_iir/rmse_noisy)*100:>9.1f}%'
)
print(
    f'{f"FIR Hamming ({numtaps_fir} tap)":<35}  '
    f'{rmse_fir:>10.4f}  {(1-rmse_fir/rmse_noisy)*100:>9.1f}%'
)

---
## 7. Pipeline Pra-Pemrosesan ECG Lengkap

Dalam praktik klinis dan penelitian, pra-pemrosesan ECG dilakukan secara bertahap
(*step-by-step pipeline*). Setiap langkah mengatasi satu jenis gangguan spesifik.

### Urutan Langkah yang Direkomendasikan

| Langkah | Operasi | Tujuan |
|---|---|---|
| **1** | Notch filter 50 Hz | Hapus interferensi jala listrik |
| **2** | Highpass filter 0.5 Hz | Hapus *baseline wander* (pernapasan) |
| **3** | Lowpass filter 40 Hz | Hapus noise frekuensi tinggi (EMG, dsb.) |

Langkah 2 dan 3 secara efektif setara dengan satu **bandpass filter 0.5--40 Hz**.
Keduanya ditampilkan terpisah agar lebih jelas kontribusi masing-masing.

> **Standar klinis (IEC 60601-2-25)**: ECG diagnostik harus memiliki bandwidth minimal
> 0.05--150 Hz; untuk monitoring rutin, 0.5--40 Hz sudah cukup.

In [ ]:
# Pipeline pra-pemrosesan ECG: tiga langkah berurutan

# Langkah 1: Hapus interferensi 50 Hz dengan notch filter
ecg_step1 = notch_filter(ecg_noisy, freq=50.0, fs=fs_ecg, quality_factor=30.0)

# Langkah 2: Hapus baseline wander dengan highpass 0.5 Hz
ecg_step2 = remove_baseline_wander(ecg_step1, fs=fs_ecg, cutoff=0.5)

# Langkah 3: Hapus noise frekuensi tinggi dengan lowpass 40 Hz
ecg_step3 = lowpass_filter(ecg_step2, cutoff=40.0, fs=fs_ecg, order=4)

# Hitung RMSE pada setiap tahap
rmse_step1 = np.sqrt(np.mean((ecg_step1 - ecg_clean) ** 2))
rmse_step2 = np.sqrt(np.mean((ecg_step2 - ecg_clean) ** 2))
rmse_step3 = np.sqrt(np.mean((ecg_step3 - ecg_clean) ** 2))

# Visualisasi setiap tahap
fig, axes = plt.subplots(5, 1, figsize=(13, 16), sharex=True)
fig.suptitle('Pipeline Pra-Pemrosesan ECG (ECG Preprocessing Pipeline)',
             fontsize=13, fontweight='bold')

plot_ecg(t_ecg, ecg_clean,
         title='[Referensi] ECG Bersih', ax=axes[0])
plot_ecg(t_ecg, ecg_noisy,
         title=f'[Input] ECG Noisy  |  RMSE = {rmse_noisy:.4f} mV', ax=axes[1])
plot_ecg(t_ecg, ecg_step1,
         title=f'[Langkah 1] Setelah Notch 50 Hz  |  RMSE = {rmse_step1:.4f} mV',
         ax=axes[2])
plot_ecg(t_ecg, ecg_step2,
         title=(
             f'[Langkah 2] Setelah Highpass 0.5 Hz (hapus baseline)  '
             f'|  RMSE = {rmse_step2:.4f} mV'
         ),
         ax=axes[3])
plot_ecg(t_ecg, ecg_step3,
         title=(
             f'[Langkah 3] Setelah Lowpass 40 Hz (output akhir)  '
             f'|  RMSE = {rmse_step3:.4f} mV'
         ),
         ax=axes[4])

for ax in axes:
    ax.set_xlim(0, 6)

plt.tight_layout()
plt.show()

# Ringkasan RMSE dengan progress bar ASCII
print('\n--- RMSE di Setiap Tahap Pipeline ---')
stages = [
    ('ECG Noisy (input)',             rmse_noisy),
    ('Setelah Notch 50 Hz',           rmse_step1),
    ('Setelah Highpass 0.5 Hz',       rmse_step2),
    ('Setelah Lowpass 40 Hz (akhir)', rmse_step3),
]
for nama, rmse in stages:
    pct = (1 - rmse / rmse_noisy) * 100
    bar = '#' * max(0, int(pct / 3))
    print(f'  {nama:<35} RMSE={rmse:.4f}  [{bar:<30}] {pct:5.1f}%')

---
## 8. Ekstraksi Pita Frekuensi EEG (*EEG Band Extraction*)

Salah satu aplikasi terpenting filter bandpass dalam neurologi adalah **dekomposisi EEG**
ke dalam pita-pita frekuensi fisiologis. Setiap pita berkaitan dengan kondisi kognitif
dan neurologis yang spesifik.

| Pita (*Band*) | Rentang (Hz) | Kondisi Terkait |
|---|---|---|
| **Delta** ($\delta$) | 0.5 -- 4 | Tidur nyenyak, anestesi dalam |
| **Theta** ($\theta$) | 4 -- 8 | Mengantuk, meditasi, memori |
| **Alpha** ($\alpha$) | 8 -- 13 | Relaksasi, mata tertutup |
| **Beta** ($\beta$) | 13 -- 30 | Konsentrasi, kognitif aktif |

Dengan `bandpass_filter` dari `medsinyal`, kita bisa mengekstraksi setiap pita secara
langsung, lalu membandingkan hasilnya dengan *ground truth* dari data sintetis.

In [ ]:
# Definisi pita frekuensi EEG
eeg_bands = {
    'delta': (0.5, 4.0),
    'theta': (4.0, 8.0),
    'alpha': (8.0, 13.0),
    'beta':  (13.0, 30.0),
}

# Ekstraksi pita menggunakan bandpass_filter dari medsinyal
extracted_bands = {}
for band_name, (fL, fH) in eeg_bands.items():
    extracted_bands[band_name] = bandpass_filter(
        eeg_noisy, lowcut=fL, highcut=fH, fs=fs_eeg, order=4
    )
    print(f'Ekstraksi pita {band_name:<6} ({fL:4.1f}--{fH:5.1f} Hz) selesai.')

# Ground truth dari data sintetis
gt_bands = {
    'delta': data_eeg['band_delta'],
    'theta': data_eeg['band_theta'],
    'alpha': data_eeg['band_alpha'],
    'beta':  data_eeg['band_beta'],
}

In [ ]:
# Visualisasi: pita yang diekstraksi vs ground truth
band_colors = {
    'delta': '#1f77b4',
    'theta': '#ff7f0e',
    'alpha': '#2ca02c',
    'beta':  '#d62728',
}

fig, axes = plt.subplots(4, 1, figsize=(13, 14), sharex=True)
fig.suptitle('Perbandingan Pita EEG yang Diekstraksi vs Ground Truth',
             fontsize=13, fontweight='bold')

rmse_bands = {}
for i, (band_name, (fL, fH)) in enumerate(eeg_bands.items()):
    color = band_colors[band_name]
    gt    = gt_bands[band_name]
    extr  = extracted_bands[band_name]
    rmse  = np.sqrt(np.mean((extr - gt) ** 2))
    rmse_bands[band_name] = rmse

    axes[i].plot(t_eeg, gt,   color=color, linewidth=0.8, alpha=0.5,
                 label='Ground Truth')
    axes[i].plot(t_eeg, extr, color=color, linewidth=1.2, linestyle='--',
                 label=f'Diekstraksi (RMSE={rmse:.3f} uV)')
    axes[i].set_ylabel('Amplitudo (uV)')
    axes[i].set_title(f'Pita {band_name.capitalize()} ({fL}--{fH} Hz)')
    axes[i].legend(loc='upper right', fontsize=9)

axes[-1].set_xlabel('Waktu (s)')
plt.tight_layout()
plt.show()

# Ringkasan RMSE per pita
print('\n--- RMSE Ekstraksi Pita EEG vs Ground Truth ---')
print(f'{"Pita":<8} {"Rentang (Hz)":<16} {"RMSE (uV)":>10}')
print('-' * 36)
for band_name, (fL, fH) in eeg_bands.items():
    print(f'{band_name.capitalize():<8} {str(fL)+"--"+str(fH)+" Hz":<16} '
          f'{rmse_bands[band_name]:>10.4f}')
print('\nPengamatan: RMSE kecil menunjukkan filter bandpass berhasil')
print('mengekstraksi pita frekuensi yang sesuai dengan sinyal ground truth.')

In [ ]:
# Visualisasi Power Spectral Density (PSD) untuk memverifikasi ekstraksi pita
fig, axes = plt.subplots(2, 1, figsize=(13, 8))
fig.suptitle('Spektrum Daya EEG: Sebelum dan Sesudah Ekstraksi Pita',
             fontsize=13, fontweight='bold')

# PSD dari EEG noisy (input)
f_psd, psd_noisy = sig.welch(eeg_noisy, fs=fs_eeg, nperseg=512)
_, psd_clean     = sig.welch(eeg_clean,  fs=fs_eeg, nperseg=512)
axes[0].semilogy(f_psd, psd_noisy, 'k-',    linewidth=1.2, label='EEG Noisy')
axes[0].semilogy(f_psd, psd_clean,  'gray', linewidth=0.8,
                 linestyle='--', alpha=0.6, label='EEG Bersih')

# Arsir area setiap pita frekuensi
for bname, (fL, fH) in eeg_bands.items():
    axes[0].axvspan(fL, fH, alpha=0.15, color=band_colors[bname],
                    label=bname.capitalize())

axes[0].set_xlim(0, 50)
axes[0].set_xlabel('Frekuensi (Hz)')
axes[0].set_ylabel('PSD (uV^2/Hz)')
axes[0].set_title('PSD EEG Noisy dengan Pita Frekuensi yang Ditandai')
axes[0].legend(loc='upper right', fontsize=8, ncol=2)

# PSD dari setiap pita yang diekstraksi
for bname, (fL, fH) in eeg_bands.items():
    color  = band_colors[bname]
    f_b, psd_b = sig.welch(extracted_bands[bname], fs=fs_eeg, nperseg=512)
    axes[1].semilogy(f_b, psd_b, color=color, linewidth=1.2,
                     label=f'{bname.capitalize()} ({fL}--{fH} Hz)')

axes[1].set_xlim(0, 50)
axes[1].set_xlabel('Frekuensi (Hz)')
axes[1].set_ylabel('PSD (uV^2/Hz)')
axes[1].set_title('PSD Masing-Masing Pita Setelah Diekstraksi')
axes[1].legend(loc='upper right', fontsize=9)

plt.tight_layout()
plt.show()

print('Setiap pita yang diekstraksi hanya mengandung energi pada rentang frekuensinya,')
print('mengkonfirmasi bahwa filter bandpass bekerja dengan benar.')

---
## 9. Kesimpulan

### Ringkasan Materi Minggu 4

1. **Filter digital adalah alat esensial** dalam pengolahan sinyal biomedis. Berbeda
   dengan *moving average*, filter frekuensi-selektif memungkinkan kita menghilangkan
   noise tertentu tanpa mendistorsi fitur klinis yang penting.

2. **Respons frekuensi** $H(f)$ sepenuhnya menggambarkan karakteristik filter. Respons
   magnitudo menunjukkan frekuensi yang dilewatkan atau diblokir; respons fase menunjukkan
   distorsi temporal. Orde filter yang lebih tinggi menghasilkan rolloff lebih tajam.

3. **Filter IIR Butterworth** bersifat rekursif (menggunakan umpan balik), efisien secara
   komputasi, dengan passband maksimum datar. Cocok untuk sistem *real-time* dan *embedded*.
   Digunakan luas di peralatan ECG dan EEG klinis.

4. **Notch filter** adalah alat yang tepat sasaran untuk menghilangkan interferensi jala
   listrik 50/60 Hz. Parameter *quality factor* $Q$ mengontrol seberapa sempit notch --
   $Q$ tinggi meminimalkan dampak pada frekuensi sinyal yang berdekatan.

5. **Filter FIR** bersifat non-rekursif, selalu stabil, dan memiliki fase linear (tidak ada
   distorsi fase). Meski membutuhkan lebih banyak koefisien dibanding IIR, FIR lebih cocok
   untuk aplikasi yang memerlukan presisi temporal tinggi.

6. **Pipeline pra-pemrosesan ECG** yang lengkap terdiri dari tiga tahap berurutan:
   notch 50 Hz $\to$ highpass 0.5 Hz $\to$ lowpass 40 Hz. Pendekatan bertahap ini
   memudahkan debugging dan memungkinkan evaluasi kontribusi setiap tahap.

### Minggu Depan

Kita akan mempelajari secara mendalam **Sinyal ECG dan Deteksi QRS** (*ECG Signal and
QRS Detection*). Dengan bekal filter digital dari minggu ini, kita akan membangun algoritma
untuk mendeteksi R-peak secara otomatis menggunakan metode Pan-Tompkins -- fondasi dari
analisis denyut jantung (*heart rate analysis*) dan diagnosis aritmia.

---

### Referensi

1. Rangayyan, R.M. *Biomedical Signal Analysis*, 2nd Ed. Wiley-IEEE Press, 2015.
   (Chapter 6: Filtering)
2. Sörnmo, L. & Laguna, P. *Bioelectrical Signal Processing in Cardiac and Neurological
   Applications*. Academic Press, 2005. (Chapter 3: Digital Filters)
3. Proakis, J.G. & Manolakis, D.G. *Digital Signal Processing: Principles, Algorithms,
   and Applications*, 4th Ed. Pearson, 2007. (Chapters 7--8: IIR and FIR Filter Design)
4. Pan, J. & Tompkins, W.J. "A real-time QRS detection algorithm."
   *IEEE Transactions on Biomedical Engineering*, 32(3):230--236, 1985.